In [6]:
! pip install PyStemmer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyStemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=579732 sha256=0082cd0deb2708918a4f2ba69ebcf340db753cbf05090bf71235e5302c1086d9
  Stored in directory: /root/.cache/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built PyStemmer


In [7]:
import sys
import numpy as np
import pickle
import re
from Stemmer import Stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [8]:
# очистка текста с помощью regexp приведение слов в инфинитив и нижний регистр, замена цифр

def text_cleaner(text):
    text = text.lower() # приведение в lowercase
    stemmer = Stemmer('russian')
    text = ' '.join( stemmer.stemWords( text.split() ) )
    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр
    return  text

In [9]:
# загрузка данных из файла model.txt

def load_data():
    data = {'text':[],'tag':[]}
    for line in open('model.txt'):
        if not('#' in line):
            row = line.split("@")
            data['text'] += [row[0]]
            data['tag'] += [row[1]]
    return data

In [10]:
# Обучение

def train_test_split(data, validation_split = 0.1):
    sz = len(data['text'])
    indices = np.arange(sz)
    np.random.shuffle(indices)

    X = [data['text'][i] for i in indices]
    Y = [data['tag'][i] for i in indices]
    nb_validation_samples = int( validation_split * sz )

    return {
        'train': {'x': X[:-nb_validation_samples], 'y': Y[:-nb_validation_samples]},
        'test': {'x': X[-nb_validation_samples:], 'y': Y[-nb_validation_samples:]}
    }

In [13]:
def openai():
    data = load_data()
    D = train_test_split(data)
    text_clf = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', SGDClassifier(loss='hinge')),
                    ])
    text_clf.fit(D['train']['x'], D['train']['y'])
    predicted = text_clf.predict( D['train']['x'] )

# Тестирование программы

    z = input("Введите вопрос без знака вопроса на конце: ")
    zz = []
    zz.append(z)
    predicted = text_clf.predict(zz)
    print(predicted[0])

In [15]:
if __name__ == '__main__':
    sys.exit(openai())

Введите вопрос без знака вопроса на конце: Почему небо голубое
 Информация о предмете



SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
